In [2]:
import sys
import os
import warnings
sys.path.append("../db")

if not sys.warnoptions:
    warnings.simplefilter("ignore")

from database import init_db, db_session
from models import Metadata, Output, Parameters

import pandas as pd
import geopandas as gpd
import numpy as np
import shapely.geometry
import json
import yaml
import configparser
import redis
import boto3
from datetime import datetime
from collections import OrderedDict
from hashlib import sha256
import urllib.request
import shutil
import time
import glob

import random
from shapely.ops import cascaded_union
from shapely.geometry import Point

postgres


In [68]:
df = pd.read_csv('FSC.csv')

In [69]:
df = df[['P0','R0','dR','dC','S0','Region','Shock','Country']]

In [72]:
df = df.rename(columns={'Region': "shocked_region", "Shock": "shock_severity", "Country": "country"})

In [75]:
df = df.replace('MBBF','ALL')

In [87]:
def generate_impact_level(row):
    if row.dR == 0 and row.dC == 0:
        return "No Impact"
    elif row.dR < 0 and abs(row.dR)<row.R0 and row.dC==0:
        return "Low Impact"
    elif (row.R0 + row.dR) == 0 and (row.dR + row.dC)/row.S0 > -0.01:
        return "Medium Impact"
    elif (row.R0 + row.dR) == 0 and (row.dR + row.dC)/row.S0 < -0.01:
        return "Serious Impact"

In [88]:
df['impact_level'] = df.apply(lambda row: generate_impact_level(row),axis=1)

In [91]:
df = df[['country','shocked_region','shock_severity','impact_level']]

In [93]:
df.to_csv("FSC_formatted.csv", index=False)